In [1]:
import numpy as np
import pandas as pd
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.manifold import TSNE
import nltk as nlp
nltk.download("stopwords")

C:\Users\oktay\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oktay\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
tweets = pd.read_csv("tweets.csv")


#Reading the datas that we classified.

In [3]:
list(tweets.columns.values)

['tweet_id',
 'airline_sentiment',
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'airline_sentiment_gold',
 'name',
 'negativereason_gold',
 'retweet_count',
 'text',
 'tweet_coord',
 'tweet_created',
 'tweet_location',
 'user_timezone']

In [4]:
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
sentiment_counts = tweets.airline_sentiment.value_counts()
number_of_tweets = tweets.tweet_id.count()
print(sentiment_counts)

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64


It's not a equal dataset. %62.7 tweet was negative. That clearly wouldn't be a useful classifier, but worth to remember.


In [6]:
#Cleaning the entries from non alphabetical values and numerical values.
text_list = []
for text in tweets.text:
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ",text) # a-z dışındakileri boşlukla değiştir
    text =nltk.word_tokenize(text)
    text =[word for word in text if not word in set(stopwords.words("English"))]
    lemma = nlp.WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    text = " ".join(text)
    text_list.append(text)
text_list

['virginamerica dhepburn said',
 'virginamerica plus added commercial experience tacky',
 'virginamerica today must mean need take another trip',
 'virginamerica really aggressive blast obnoxious entertainment guest face amp little recourse',
 'virginamerica really big bad thing',
 'virginamerica seriously would pay flight seat playing really bad thing flying va',
 'virginamerica yes nearly every time fly vx ear worm go away',
 'virginamerica really missed prime opportunity men without hat parody http co mwpg grezp',
 'virginamerica well',
 'virginamerica amazing arrived hour early good',
 'virginamerica know suicide second leading cause death among teen',
 'virginamerica lt pretty graphic much better minimal iconography',
 'virginamerica great deal already thinking nd trip australia amp even gone st trip yet p',
 'virginamerica virginmedia flying fabulous seductive sky u take stress away travel http co ahlxhhkiyn',
 'virginamerica thanks',
 'virginamerica sfo pdx schedule still mia',


In [7]:
print(text_list[:50])

['virginamerica dhepburn said', 'virginamerica plus added commercial experience tacky', 'virginamerica today must mean need take another trip', 'virginamerica really aggressive blast obnoxious entertainment guest face amp little recourse', 'virginamerica really big bad thing', 'virginamerica seriously would pay flight seat playing really bad thing flying va', 'virginamerica yes nearly every time fly vx ear worm go away', 'virginamerica really missed prime opportunity men without hat parody http co mwpg grezp', 'virginamerica well', 'virginamerica amazing arrived hour early good', 'virginamerica know suicide second leading cause death among teen', 'virginamerica lt pretty graphic much better minimal iconography', 'virginamerica great deal already thinking nd trip australia amp even gone st trip yet p', 'virginamerica virginmedia flying fabulous seductive sky u take stress away travel http co ahlxhhkiyn', 'virginamerica thanks', 'virginamerica sfo pdx schedule still mia', 'virginamerica 

In [8]:
print("unique word number", len(set(text.split())))
print("total word number", len(text.split()))

unique word number 13
total word number 15


In [9]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [10]:
tmp_corpus = tweets['text'].map(lambda x: x.split('.'))

In [11]:
# corpus [[w1,w2,w3..],[..]]
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|██████████| 14640/14640 [00:00<00:00, 36721.71it/s]


In [12]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 33744
Num of words - 260444


In [13]:
phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

2020-05-25 14:53:42,243 : INFO : collecting all words and their counts
2020-05-25 14:53:42,245 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-05-25 14:53:42,427 : INFO : PROGRESS: at sentence #10000, processed 77156 words and 53473 word types
2020-05-25 14:53:42,594 : INFO : PROGRESS: at sentence #20000, processed 151011 words and 92427 word types
2020-05-25 14:53:42,749 : INFO : PROGRESS: at sentence #30000, processed 230350 words and 125667 word types
2020-05-25 14:53:42,809 : INFO : collected 137484 word types from a corpus of 260444 words (unigram + bigrams) and 33744 sentences
2020-05-25 14:53:42,811 : INFO : using 137484 counts as vocab in Phrases<0 vocab, min_count=25, threshold=50, max_vocab_size=40000000>
2020-05-25 14:53:42,812 : INFO : source_vocab length 137484
2020-05-25 14:53:44,316 : INFO : Phraser built with 28 28 phrasegrams


In [14]:
for index,sentence in enumerate(corpus):
    corpus[index]=bigram[sentence]

In [15]:
def shuffle_corpus(sentences):
    shuffled = list (sentences)
    random.shuffle(shuffled)
    return shuffled

In [16]:
size = 100 
window_size = 2 # if sentence is long we need to increase 
epochs = 100
min_count = 2 
workers = 4 
#training word2vec model using gensim
model = Word2Vec (corpus, sg=1,window=window_size,size=size,min_count=min_count,workers=workers,iter=epochs,sample=0.01)


2020-05-25 14:53:45,348 : INFO : collecting all words and their counts
2020-05-25 14:53:45,350 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-25 14:53:45,380 : INFO : PROGRESS: at sentence #10000, processed 76441 words, keeping 11696 word types
2020-05-25 14:53:45,403 : INFO : PROGRESS: at sentence #20000, processed 149501 words, keeping 19247 word types
2020-05-25 14:53:45,430 : INFO : PROGRESS: at sentence #30000, processed 227951 words, keeping 24532 word types
2020-05-25 14:53:45,443 : INFO : collected 26314 word types from a corpus of 257673 raw words and 33744 sentences
2020-05-25 14:53:45,445 : INFO : Loading a fresh vocabulary
2020-05-25 14:53:45,493 : INFO : min_count=2 retains 9396 unique words (35% of original 26314, drops 16918)
2020-05-25 14:53:45,494 : INFO : min_count=2 leaves 240755 word corpus (93% of original 257673, drops 16918)
2020-05-25 14:53:45,528 : INFO : deleting the raw counts dictionary of 26314 items
2020-05-25 14:53:45,5

2020-05-25 14:53:50,362 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:53:50,373 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:53:50,374 : INFO : EPOCH - 14 : training on 257673 raw words (239188 effective words) took 0.3s, 820630 effective words/s
2020-05-25 14:53:50,668 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:53:50,677 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:53:50,679 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:53:50,682 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:53:50,684 : INFO : EPOCH - 15 : training on 257673 raw words (239147 effective words) took 0.3s, 796020 effective words/s
2020-05-25 14:53:50,961 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:53:50,975 : INFO : worker thread finished; awaiting finish of 2 more threads


2020-05-25 14:53:55,338 : INFO : EPOCH - 30 : training on 257673 raw words (239141 effective words) took 0.4s, 555519 effective words/s
2020-05-25 14:53:55,654 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:53:55,665 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:53:55,671 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:53:55,673 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:53:55,675 : INFO : EPOCH - 31 : training on 257673 raw words (239153 effective words) took 0.3s, 730982 effective words/s
2020-05-25 14:53:55,958 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:53:55,970 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:53:55,973 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:53:55,989 : INFO : worker thread finished; awaiting finish of 0 more threads


2020-05-25 14:54:00,806 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:00,811 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:00,812 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:00,831 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:00,833 : INFO : EPOCH - 47 : training on 257673 raw words (239168 effective words) took 0.3s, 789467 effective words/s
2020-05-25 14:54:01,113 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:01,116 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:01,128 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:01,131 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:01,133 : INFO : EPOCH - 48 : training on 257673 raw words (239174 effective words) took 0.3s, 822130 effective words/s


2020-05-25 14:54:05,792 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:05,795 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:05,797 : INFO : EPOCH - 63 : training on 257673 raw words (239153 effective words) took 0.3s, 843705 effective words/s
2020-05-25 14:54:06,080 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:06,087 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:06,102 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:06,108 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:06,110 : INFO : EPOCH - 64 : training on 257673 raw words (239110 effective words) took 0.3s, 793711 effective words/s
2020-05-25 14:54:06,361 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:06,364 : INFO : worker thread finished; awaiting finish of 2 more threads


2020-05-25 14:54:10,445 : INFO : EPOCH - 79 : training on 257673 raw words (239150 effective words) took 0.3s, 860187 effective words/s
2020-05-25 14:54:10,750 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:10,761 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:10,773 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:10,775 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:10,777 : INFO : EPOCH - 80 : training on 257673 raw words (239130 effective words) took 0.3s, 746489 effective words/s
2020-05-25 14:54:11,025 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:11,031 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:11,035 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:11,040 : INFO : worker thread finished; awaiting finish of 0 more threads


2020-05-25 14:54:15,402 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:15,404 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:15,417 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:15,422 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:15,424 : INFO : EPOCH - 96 : training on 257673 raw words (239135 effective words) took 0.3s, 838269 effective words/s
2020-05-25 14:54:15,678 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-25 14:54:15,693 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-25 14:54:15,700 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-25 14:54:15,708 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-25 14:54:15,710 : INFO : EPOCH - 97 : training on 257673 raw words (239103 effective words) took 0.3s, 874808 effective words/s


In [17]:
# save model
model.save('w2v_model')

2020-05-25 14:54:16,572 : INFO : saving Word2Vec object under w2v_model, separately None
2020-05-25 14:54:16,575 : INFO : not storing attribute vectors_norm
2020-05-25 14:54:16,578 : INFO : not storing attribute cum_table
2020-05-25 14:54:16,739 : INFO : saved w2v_model


In [18]:
# load word2vec model
model = Word2Vec.load('w2v_model')

2020-05-25 14:54:16,749 : INFO : loading Word2Vec object from w2v_model
2020-05-25 14:54:16,837 : INFO : loading wv recursively from w2v_model.wv.* with mmap=None
2020-05-25 14:54:16,840 : INFO : setting ignored attribute vectors_norm to None
2020-05-25 14:54:16,842 : INFO : loading vocabulary recursively from w2v_model.vocabulary.* with mmap=None
2020-05-25 14:54:16,844 : INFO : loading trainables recursively from w2v_model.trainables.* with mmap=None
2020-05-25 14:54:16,845 : INFO : setting ignored attribute cum_table to None
2020-05-25 14:54:16,847 : INFO : loaded w2v_model


In [19]:
model.most_similar(positive=['amazing', 'costumer'], negative=['services'], topn=1)

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2020-05-25 14:54:16,881 : INFO : precomputing L2-norms of word weight vectors


[('deplorable', 0.4820473790168762)]

In [20]:
model.most_similar(positive=['amazing', 'service'], negative=['flight'], topn=5)

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('thought', 0.45611029863357544),
 ('service!!!', 0.4335615932941437),
 ('personalized', 0.4253174066543579),
 ('Awful', 0.41721048951148987),
 ('#senseless', 0.416159063577652)]

In [21]:
model.doesnt_match('hotel service flight'.split())

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
C:\Users\oktay\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:730: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'service'

In [22]:
model.doesnt_match('cheap expensive flight'.split())

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'flight'

In [23]:
model.similarity('cheap','expensive')

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.2336132142005683

In [24]:
model.most_similar('great')

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('good', 0.5530094504356384),
 ('perfect', 0.5346366167068481),
 ('excellent', 0.527241587638855),
 ('terrible', 0.5217261910438538),
 ('hoot!', 0.5083856582641602),
 ('nice', 0.5079426765441895),
 ('Fantastic', 0.501379668712616),
 ('smooth', 0.5006301403045654),
 ('Nice', 0.4822717308998108),
 ('Great', 0.48225826025009155)]

In [25]:
model.most_similar('comfortable')

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('opposed', 0.543967604637146),
 ('browser', 0.5425182580947876),
 ('difficult', 0.5321793556213379),
 ('bereavement', 0.5309464335441589),
 ('tough', 0.530639111995697),
 ('stressful', 0.5272545218467712),
 ('disgrace', 0.5272388458251953),
 ('inconvenient', 0.5213665962219238),
 ('childish', 0.5209087133407593),
 ('potentially', 0.5208833813667297)]

In [26]:
model.most_similar('recommend')

C:\Users\oktay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('Airlines!', 0.6226905584335327),
 ('adore', 0.6049330830574036),
 ('agency', 0.5935222506523132),
 ('asked,', 0.5876718759536743),
 ('Dare', 0.5854730010032654),
 ('attempted', 0.5847495794296265),
 ('swear', 0.5822001695632935),
 ('convince', 0.5670164823532104),
 ('suggest', 0.5610436201095581),
 ('protection', 0.5593390464782715)]